#### 합성곱 신경망(CNN)이란?
- CNN(Convolutional Neural Network)은 이미지 인식과 음성 인슥 등 다양한 곳에 사용되는데, 특히 이미지 인식 분야에서 딥러닝을 활용한 기법은 거의 다 CNN을 기초로 한다.
- CNN에서는 합성곱 계층(convolutional layer)과 풀링 계층(pooling layer)이 새롭게 등장한다.

#### 합성곱 계층
- 합성곱 계층이 완전연결(fully-connected) 신경망에서의 affine 계층과는 달리 데이터 형상을 유지한다는 차이점이 있다.
 * Affine 계층 : 인접하는 계층의 모든 뉴런이 결합되어 있는 계층
- 이미지와 같이 3차원의 데이터를 입력 값을 받을 경우 다음 계층에도 3차원으로 데이터를 전달하여야 하는데 CNN에서는 이미지처럼 형상을 가진 데이터를 제대로 이해할 (가능성이 있는) 수 있다.
- 합성곱 계층에서 쓰는 용어는 다음과 같다.
 * 특징 맵(feature map) : 합성곱 계층의 입출력 데이터
 * 커널(kernel) : 합성곱 연산에서 사용되는 필터를 뜻하며, 그 크기를 윈도우(window)라고 부른다.
  > 필터의 매개변수는 신경망에서의 가중치라 할 수 있다.
 * 패딩(padding) : 합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값(ex. 0)으로 채우는 것을 뜻한다.
  > 패딩은 주로 출력 크기를 조정할 목적으로 사용한다. <br>
  > 패딩의 크기를 크게하면 출력의 크기가 커진다.
 * 스트라이드(stride) : 커널을 적용하는 간격을 의미한다.
  > 스트라이드의 크기를 크게하면 출력의 크기는 커진다.

#### 합성곱 연산
1. 커널을 일정 간격으로 이동해가며 입력 데이터에 적용한다.
2. 입력과 필터에서 대응하는 원소끼리 곱합 후 그 총합을 구한다. => 단일 곱셉-누산(FMA, fuserd-multiply-add)
3. 앞선 결과를 출력의 해당 장소에 저장한다.
4. 1~3번 과정을 모든 장소에서 수행하면 합성곱 연산의 출력이 완성된다.

- 입력 크기를 (H, W), 필터의 크기를 (FH, FW), 출력 크기를 (OH, OW), 패딩을 P, 스트라이드를 S라 하면 출력 크기는 다음과 같다.
 > OH = (H+2P-FH)/S+1 <br>
 > OW = (W+2P-FW)/S+1 <br>
- 합성곱 연산 시 주의할 점은 다음과 같다.
 * 2차원 데이터의 합성곱 연산
  > 주의할 점은 출력의 크기가 정수로 나눠떨어지는 값이여야 한다.
 * 3차원 데이터의 합성곱 연산
  > 입력 데이터와 커널의 합성곱 연산을 채널마다 수행하고, 그 결과를 더해서 출력 값이 나온다. <br>
  > 따라서, 입력 데이터의 채널 수와 커널의 채널 수가 같아야 하며, 모든 채널의 필터는 같은 크기여야 한다.
 * 여러 커널을 사용한 합성곱 연산
  > 커널을 N개 적용하면 출력 값도 N개 생성된다.

#### 풀링 계층
- 풀링은 세로 & 가로 방향의 공간을 줄이는 연산이다.
- 풀링에는 최대 풀링(max pooling)과 평균 풀링(average pooling) 등이 있다.
 * 최대 풀링 : 대상 영역에서 최대 값을 취하는 연산
 * 평균 풀링 : 대상 영역의 평균을 계산
- 이미지 인식 분야에서는 주로 최대 풀링을 사용한다.

- 풀링 게층의 특징은 다음과 같다.
1. 학습해야 할 매개변수가 없다.
2. 채널 수가 변하지 않는다
3. 입력의 변화에 영향을 적게 받는다(강건하다).
 * 입력 데이터가 조금 변한다해도 풀링의 결과는 잘 변하지 않는데, 이는 입력 데이터의 차이를 풀링이 흡수해 사라지게 하기 때문이다.

#### 합성곱/풀링 계층 구현하기

- 입력 데이터

In [1]:
### 4차원 입력 데이터
import numpy as np

x = np.random.rand(10, 1, 28, 28)

print(x.shape)
print(x[0].shape)
print(x[1].shape)

(10, 1, 28, 28)
(1, 28, 28)
(1, 28, 28)


In [2]:
### im2col : 입력 데이터를 필터링(가중치 계산)하기 좋게 전개하는(펼치는) 함수
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)   # (데이터수, 채널수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


- 합성곱 계층

In [3]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T  # 필터전개
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)   # transpose: 다차원 배열의 축 순서를 바꿔주는 함수        
        return out

- 풀링 계층

In [4]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (H - self.pool-w) / self.stride)
        
        ### 1.입력 데이터를 전재한다.
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        ### 2.행 별 최댓 값을 구한다.
        out = np.max(col, axis=1)
        
        ### 3.적절한 모양으로 성형한다.
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        return out

#### CNN 구현하기

- 손글씨 숫자 이미지를 인식하는 CNN을 구현해보자.
- 초기화(__init__) 때 받는 인수는 다음과 같다.
 * input_dim : 입려 데이터(채널 수, 높이, 너비)의 차원
 * conv_param : 합성곱 계층의 하이퍼파라미터
  > - filter_num : 필터(커널) 수
  > - filter_size : 필터 크기
  > - stride : 스트라이드
  > - pad : 퓨ㅐ딩
  > - hidden_size : 은닉층(완전연결)의 뉴런 수
  > - output_size : 출력층(완전연결)의 뉴런 수
  > - weight_init_std : 초기화 때 가중치 표준편차

In [5]:
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28), conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                hidden_size=100, output_size=10, weight_init_std=0.01):
        ### 초기화 인수로 주어진 합성곱 계층의 하이퍼파라미터를 나중에 쓰기 쉽도록 꺼낸 후, 합성곱 계층의 출력 크기를 계산한다.
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size+ 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        ### 가중치 매개변수를 초기화한다.
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        ### CNN을 구성하는 계층들 생성한다.
        self.layers = OrderedDIct()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
        slef.layers['ReLU1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['ReLU2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layers = SoftmaxWithLoss()
        
    ### 추론을 수행하는 메서드 구현
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    ### 손실 함수의 값을 구하는 loss 매서드 구현
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    ### 오차역전파법으로 기울기 구하는 메서드 구현
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        
        return grads

#### CNN 시각화하기
- 합성곱 계층의 가중치 : 가중치의 원소는 실수이지만, 이미지에서는 가장 작은 값(0)은 검은색, 가장 큰 값(255)은 흰색으로 정규화하여 표시한다.
- 학습을 마친 합성곱 계층의 필터는 규칙성 있는 필터로 바뀐다.
 * 규칙성 있는 필터에는 에지나 블롭 등이 있으며, 이는 원시적인 정보를 추출할 수 있다.
  > 에지 : 색상이 바뀐 경계선 <br>
  > 블롭(blob) : 국소적으로 덩어리진 영역